<a href="https://colab.research.google.com/github/SophieShin/DeepLearning/blob/main/%5BAIM5004%5DAssignment_3(SoheeShin).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment #3
### Training a character-level language model with recurrent neural networks and Transformers architecture. 


In [1]:
import tensorflow as tf

import os
import time

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#from utils import  forward_tracer, backward_tracer, Char2Vec, num_flat_features

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np

from tqdm import tqdm

from IPython.display import clear_output

import string
import random
import re

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

### (a) Download shakespeare dataset from https://storage.googleapis.com/download. tensorflow.org/data/shakespeare.txt. Report the number of unique characters and this number will be the number of your vocabulary (note that ’a’ and ’A’ are different characters). Also, show 3 random chunks (200 chracters per each) of the dataset.

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
len(text)

1115394

In [ ]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [5]:
vocab = sorted(set(text))
print(vocab)
print ('Number of Unique Characters: {}'.format(len(vocab)))

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Number of Unique Characters: 65


In [6]:
chunk_len = 200

def random_chunk():
    start_index = random.randint(0,len(text) - chunk_len)
    end_index = start_index + chunk_len + 1
    return text[start_index:end_index]

print(random_chunk())

f,
Shall do and undo as him pleaseth best.

RICHARD:
Let me be Duke of Clarence, George of Gloucester;
For Gloucester's dukedom is too ominous.

WARWICK:
Tut, that's a foolish observation:
Richard, be 


###  (2 pts) Design a vanila RNN architecture and write the training codes w/ following hyperparameters. Report the number of model parameters.

In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
seq_length = 200
examples_per_epoch = len(text)//seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(3):
  print(idx2char[i.numpy()])

F
i
r


In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you '
"know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be"
' done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but'
' the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to parti'
'cularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak thi

In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for input_example, target_example in  dataset.take(1):
  print ('Input: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target: ', repr(''.join(idx2char[target_example.numpy()])))

Input:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you'
Target:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you '


In [12]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("{:4d}".format(i))
    print("  Input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  Expected Output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

   0
  Input: 18 ('F')
  Expected Output: 47 ('i')
   1
  Input: 47 ('i')
  Expected Output: 56 ('r')
   2
  Input: 56 ('r')
  Expected Output: 57 ('s')
   3
  Input: 57 ('s')
  Expected Output: 58 ('t')
   4
  Input: 58 ('t')
  Expected Output: 1 (' ')


In [13]:
BATCH_SIZE = 64

BUFFER_SIZE = 1000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 200), dtype=tf.int64, name=None), TensorSpec(shape=(64, 200), dtype=tf.int64, name=None))>

In [14]:
vocab_size = len(vocab)

embedding_dim = 64

rnn_units = 1024

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.SimpleRNN(rnn_units,
                                 return_sequences=True,
                                 stateful=True,
                                 recurrent_initializer='glorot_uniform'),
        tf.keras.layers.SimpleRNN(rnn_units,
                                 return_sequences=True,
                                 stateful=True,
                                 recurrent_initializer='glorot_uniform'),
        tf.keras.layers.SimpleRNN(rnn_units,
                                 return_sequences=True,
                                 stateful=True,
                                 recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)        
    ])
    return model

In [16]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [17]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (Batch Size, Length of Sequence, Vocab Size)")

(64, 200, 65) # (Batch Size, Length of Sequence, Vocab Size)


In [18]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 64)            4160      
                                                                 
 simple_rnn (SimpleRNN)      (64, None, 1024)          1115136   
                                                                 
 simple_rnn_1 (SimpleRNN)    (64, None, 1024)          2098176   
                                                                 
 simple_rnn_2 (SimpleRNN)    (64, None, 1024)          2098176   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,382,273
Trainable params: 5,382,273
Non-trainable params: 0
_________________________________________________________________


In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("expected batch(shape): ", example_batch_predictions.shape)
print("Loss:          ", example_batch_loss.numpy().mean())

expected batch(shape):  (64, 200, 65)
Loss:           4.32143


In [20]:
model.compile(optimizer='adam', loss=loss)
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [21]:
EPOCHS=30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
86/86 [==============================] - 48s 503ms/step - loss: 4.0286
Epoch 2/30
86/86 [==============================] - 44s 510ms/step - loss: 3.3281
Epoch 3/30
86/86 [==============================] - 44s 507ms/step - loss: 3.3236
Epoch 4/30
86/86 [==============================] - 44s 506ms/step - loss: 3.3203
Epoch 5/30
86/86 [==============================] - 43s 501ms/step - loss: 3.3186
Epoch 6/30
86/86 [==============================] - 43s 493ms/step - loss: 3.1490
Epoch 7/30
86/86 [==============================] - 43s 494ms/step - loss: 2.7718
Epoch 8/30
86/86 [==============================] - 43s 493ms/step - loss: 2.5383
Epoch 9/30
86/86 [==============================] - 43s 503ms/step - loss: 2.3829
Epoch 10/30
86/86 [==============================] - 42s 488ms/step - loss: 2.2487
Epoch 11/30
86/86 [==============================] - 43s 493ms/step - loss: 2.1420
Epoch 12/30
86/86 [==============================] - 43s 500ms/step - loss: 2.0556
Epoch 13/30
8

In [22]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [23]:
def generate_text(model, start_string):

  num_generate = 200

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [24]:
print(generate_text(model, start_string=u"We are "))

We are unsider?
My dotten hate to them follow.

BIANCA:
Anot, fair hath by me on enture it catent.

TRANIO:
Bo, there strant notle
Which nage their courtep, to be much the cames brought to death dut of to en


In [ ]:
print(generate_text(model, start_string=u"what "))

what may canch delly:
'Tis but the priase is his a glory.
Lo, is he so,
For he did lack to pass my top again,
For 'tis another, again
Afon of poor of prile. Now, my hastily your fellow she:
Great you the t


In [ ]:
print(generate_text(model, start_string=u"You "))

You knock am I.

TRANIO:
Good friar, I know; and then you are warmed, sir:
Have I not too much of you?

PAULINA:
Trust me, I will follow
Our uncles of rebellion, the queen too must
Claude here from Burgut


In [ ]:
print(generate_text(model, start_string=u"I tell you, friends "))

I tell you, friends with his soul both
to her.

Clown:
Please you, madam:
I am come to rest thou without fortune.

BUCKINGHAM:
Atay, he died is grief and pack of file.

GRUMIO:
Of all would pick
The penture of his chambe


###(c)  Perplexity is defined as the exponentiated average negative log-likelihood of a sequence. Train your network RNNs and provide a PPL curve over the course of the training.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(nn.NLLLoss)

### (d)  Among GRU, LSTM, or Transformer, pick one of your favorite architecture, and design the architecture whose the number of parameters is similar to vanila RNN you implemented above. Then train and provide a PPL curve over the course of the training (in the same plots in (c)). You are free to select any hyperparameters if needed (no need to use the hyperparameters above). Report the number of model parameters. 

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape)

(64, 200, 65)


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Expected batch(shape): ", example_batch_predictions.shape)
print("Loss:          ", example_batch_loss.numpy().mean())

Expected batch(shape):  (64, 200, 65)
Loss:           4.174212


In [ ]:
model.compile(optimizer='adam', loss=loss)


In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS=30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
86/86 [==============================] - 9s 102ms/step - loss: 2.9079
Epoch 2/30
86/86 [==============================] - 9s 107ms/step - loss: 2.3516
Epoch 3/30
86/86 [==============================] - 10s 109ms/step - loss: 2.1735
Epoch 4/30
86/86 [==============================] - 10s 109ms/step - loss: 2.0051
Epoch 5/30
86/86 [==============================] - 9s 106ms/step - loss: 1.8606
Epoch 6/30
86/86 [==============================] - 9s 102ms/step - loss: 1.7406
Epoch 7/30
86/86 [==============================] - 9s 101ms/step - loss: 1.6436
Epoch 8/30
86/86 [==============================] - 9s 101ms/step - loss: 1.5676
Epoch 9/30
86/86 [==============================] - 9s 101ms/step - loss: 1.5055
Epoch 10/30
86/86 [==============================] - 9s 102ms/step - loss: 1.4546
Epoch 11/30
86/86 [==============================] - 9s 102ms/step - loss: 1.4127
Epoch 12/30
86/86 [==============================] - 9s 102ms/step - loss: 1.3764
Epoch 13/30
86/86 [====

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):

  num_generate = 200

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

###(e) Pick the best performing (lowest PPL score) model, and generate the text autoregressively given the following prompts.


In [ ]:
print(generate_text(model, start_string=u"We are "))

We are malapedly.
One--to foolish weed inhustice, howling
That I am put to doubt, that you have wounded him, who is
not King of nuster your thoughts,
Which makes her the curting fatal treasure,
The lies worm


In [ ]:
print(generate_text(model, start_string=u"what "))

what now I may cold for me
to be the battle king you woo a concement,
and, by the sea, but yield in darnest. So his goodly estate we are gone
To prattle to mine unlew my cup drunk.
I hope to her and jots n


In [ ]:
print(generate_text(model, start_string=u"You "))

You that he return?
Our king and undertain'st armade, and ebrthes,
And bid me see withall purpose and revoke
As man have matesimely took to do.

HORTENSIO:
My assailing is double!

SEBASTIAN:
He is it tea


In [ ]:
print(generate_text(model, start_string=u"I tell you, friends "))

I tell you, friends well down.

DUKE VINCENTIO:
This shall denie the wife to accomplimatoon;
A single toothinks peace encounter dawly heaven
But to injects night: look you then well beget, here was full alone
Thou thrive
